<a href="https://colab.research.google.com/github/INA-95/H-M-Personalized-Fashion-Recommendations/blob/main/Movie_Recommender_System_230301.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import tqdm
from typing import Dict, List, Any

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Project/Side_Project/H&M_recommendation_system/data

Mounted at /content/gdrive
/content/gdrive/MyDrive/Project/Side_Project/H&M_recommendation_system/data


In [3]:
from glob import glob
files = glob('./*.csv')

In [4]:
dfs = {file:pd.read_csv(file) for file in files}
dfs

{'./df_small.csv':          Unnamed: 0  Unnamed: 0.1  userId  movieId  rating  \
 0               960           960       5        1     4.5   
 1               961           961       5       10     2.5   
 2               962           962       5       19     3.5   
 3               963           963       5       32     5.0   
 4               964           964       5       39     4.5   
 ...             ...           ...     ...      ...     ...   
 5392020    19998291      19998291    2704     4993     5.0   
 5392021    19998292      19998292    2704     5349     3.0   
 5392022    19998293      19998293    2704     5378     4.0   
 5392023    19998295      19998295    2704     5449     4.0   
 5392024    19998296      19998296    2704     5459     4.0   
 
                    timestamp  movie_id_idx  
 0        2009-01-02 01:13:41             0  
 1        2009-01-02 01:15:59             9  
 2        2009-01-01 04:21:44            18  
 3        2009-01-01 04:11:35           

In [5]:
df = dfs['./df_small.csv']

In [6]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'movieId', 'timestamp'], axis = 1)

,userId,rating,movie_id_idx
0,5,4.5,0
1,5,2.5,9
2,5,3.5,18
3,5,5.0,29
4,5,4.5,33
...,...,...,...
5392020,2704,5.0,1730
5392021,2704,3.0,1769
5392022,2704,4.0,1772
5392023,2704,4.0,1780


In [7]:
df = df[['userId', 'movie_id_idx', 'rating']]

In [8]:
# Information of Dataset

num_of_users = df['userId'].nunique()
num_of_movies = df['movie_id_idx'].nunique()
print('num_of_users:', num_of_users)
print('num_of_movies:', num_of_movies)

num_of_users: 10000
num_of_movies: 2000


In [9]:
# Split into train and test

train, test = train_test_split(df, test_size = 0.2)

In [10]:
# user_movie = {user_id : [movie1, movie2, ...]}
# movie_user = {movie_idx : [user1, user2, ...]}
# user_movie_rating = {(user, movie) : rating, (user2, movie2) : rating, ...}

def create_dict_1(target_1:List) -> Dict:
    res_dict_1 = defaultdict(list)
    for v1, v2 in target_1:
        res_dict_1[v1].append(v2)
    return res_dict_1

def create_dict_2(target_2:zip()) -> Dict:
    res_dict_2 = defaultdict(int)
    for v1, v2, v3 in target_2:
        res_dict_2[(v1, v2)] = v3
    return res_dict_2

In [11]:
user_movie = list(zip(df['userId'], df['movie_id_idx']))
movie_user = list(zip(df['movie_id_idx'], df['userId']))
user_movie_rating = zip(df['userId'], df['movie_id_idx'], df['rating'])

user_movie_dict = create_dict_1(user_movie)
movie_user_dict = create_dict_1(movie_user)
user_movie_rating_dict = create_dict_2(user_movie_rating)

In [19]:
train['user_movie'] = train['userId'].apply(lambda x : user_movie_dict[x])

In [21]:
train['movie_user'] = train['movie_id_idx'].apply(lambda x : movie_user_dict[x])

In [22]:
train

,userId,movie_id_idx,rating,user_movie,movie_user
4129456,2588,205,2.0,"[0, 1, 21, 29, 40, 43, 81, 84, 85, 96, 100, 10...","[5, 10, 15, 27, 39, 44, 56, 78, 86, 92, 95, 97..."
2161973,7096,221,3.0,"[0, 1, 4, 5, 9, 15, 18, 22, 24, 29, 30, 33, 42...","[5, 10, 15, 17, 27, 30, 39, 44, 47, 56, 78, 86..."
1462376,1585,1008,4.0,"[0, 5, 13, 15, 17, 20, 22, 28, 29, 30, 31, 33,...","[5, 30, 39, 44, 78, 92, 116, 119, 139, 154, 15..."
2284870,8081,983,3.0,"[0, 1, 2, 12, 5, 6, 8, 9, 10, 11, 13, 15, 16, ...","[15, 17, 27, 33, 47, 56, 78, 86, 102, 115, 119..."
3189360,5209,681,5.0,"[0, 2, 9, 16, 20, 30, 40, 74, 81, 84, 85, 101,...","[10, 15, 17, 27, 30, 39, 44, 47, 56, 78, 86, 8..."
...,...,...,...,...,...
3036158,4108,1368,4.0,"[0, 1, 15, 18, 20, 27, 29, 40, 80, 84, 85, 100...","[39, 88, 102, 122, 139, 192, 199, 226, 266, 26..."
2912585,3233,681,3.5,"[0, 1, 28, 29, 33, 40, 41, 43, 85, 96, 99, 100...","[10, 15, 17, 27, 30, 39, 44, 47, 56, 78, 86, 8..."
5075398,9758,397,3.0,"[0, 1, 9, 11, 14, 17, 20, 21, 27, 29, 30, 33, ...","[10, 17, 27, 39, 56, 78, 86, 100, 102, 115, 14..."
5149626,305,1074,4.0,"[0, 1, 2, 4, 5, 8, 9, 14, 15, 17, 18, 19, 20, ...","[27, 78, 95, 116, 119, 133, 161, 209, 248, 259..."
